Prepare Database

In [9]:
#General
import os

#Database Connection
import mysql.connector

#Web Scraping
import requests
from bs4 import BeautifulSoup

In [10]:
#Connect to the database
db = mysql.connector.connect(
    host="localhost",
    user="user",
    password="password",
    port="3306",
    database="realestatepredictor"
)

#Check if the table homegate exists. Set up the table if it does not exist
mycursor = db.cursor()
mycursor.execute("SHOW TABLES")
tables = mycursor.fetchall()
if ('homegate',) in tables:
    print("Table homegate exists")
else:
    print("Table homegate does not exist. Setting up table.")
    mycursor.execute("CREATE TABLE homegate (id INT AUTO_INCREMENT PRIMARY KEY, title VARCHAR(255), price FLOAT, rooms FLOAT, floor INT, address VARCHAR(255), url VARCHAR(255))")

Table homegate does not exist. Setting up table.


Homegate

In [11]:
# Define the URL to scrape
url = "https://www.homegate.ch/rent/real-estate/city-zurich/matching-list?tab=list&o=sortToplisting-desc"

# Send a GET request to the URL and get the HTML content
response = requests.get(url)
html_content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find all the listings on the page
listings = soup.find_all("div", {"class": "ListItemContainer-sc-1i7e9a5-0"})

# Create a cursor object to execute SQL queries
cursor = db.cursor()

# Iterate over each listing
for listing in listings:
    # Get the listing price
    price = listing.find("div", {"class": "ListItemPrice-sc-1i7e9a5-2"}).text.strip()

    # Get the listing m2
    m2 = listing.find("div", {"class": "ListItemArea-sc-1i7e9a5-3"}).text.strip()

    # Get the listing number of rooms
    rooms = listing.find("div", {"class": "ListItemRooms-sc-1i7e9a5-4"}).text.strip()

    # Get the listing floor
    floor = listing.find("div", {"class": "ListItemFloor-sc-1i7e9a5-5"}).text.strip()

    # Get the listing address
    address = listing.find("div", {"class": "ListItemAddress-sc-1i7e9a5-6"}).text.strip()

    # Define the SQL query to insert the data into the database
    sql = f"INSERT INTO listings (price, m2, rooms, floor, address) VALUES ('{price}', '{m2}', '{rooms}', '{floor}', '{address}')"

    # Execute the SQL query
    cursor.execute(sql)

# Commit the changes to the database
db.commit()

# Print a success message
print(f"{len(listings)} listings were successfully scraped and saved to the database.")

0 listings were successfully scraped and saved to the database.
